# Implementing LeNet-5 with Keras

I will be implementing the LeNet-5 CNN architecture with Keras in python. I will not be using TensorFlow backend since my machine has an AMD GPU, thus i will be using PlaidML backend. I will also train the model on CPU just to show how bad it is. The dataset will be CIFAR-10 which has 60k images for 10 classes.

## PlaidML as backend instead of TensorFlow

To use PlaidML as backend follow these steps
- activate project environment
- `plaidml-setup`
- `Enable experimental device support? (y,n)[n]:n`
- `Please choose a default device:` choose 1,2,3.. most probably 1 would be your CPU, 2 would be the integrated GPU of your CPU and 3 would be your dedicated GPU
- `Save settings to /Users/veersingh/.plaidml? (y,n)[y]:y`

In [1]:
# import libraries and set backend to plaidML
from os import environ
environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.preprocessing.image import ImageDataGenerator
import time
from keras import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

Using plaidml.keras.backend backend.


## Processing the data

First we specify the paths to the directories containing our training and testing sets. Next we will rescale the images from 0-255 to 0-1 to reduce computations. Then we will use the flow_from_directory() method of the ImageDataGenerator class to resize the image to 32x32x1. This is important since LeNet-5 expects the input image to be of size 32x32 with single channel or grayscale. Our input images are of the size 32x32x3 which means we will keep the image width and height the same but change rgb to grayscale.

In [2]:
# Specify the paths
train_dataset_path = '/Users/veersingh/Desktop/cifar_10/train'
test_dataset_path = '/Users/veersingh/Desktop/cifar_10/test'

In [3]:
# Rescale
train = ImageDataGenerator(rescale=1 / 255)
test = ImageDataGenerator(rescale=1 / 255)

# Define train and test dataset
# This method is useful since our images are in their respective class folders
train_dataset = train.flow_from_directory(seed=1, # random seed to maintain same output everytime
                                          directory=train_dataset_path, # path to train set
                                          target_size=(32, 32), # resize to 32x32
                                          batch_size=32, # can be 32, 64, 128
                                          class_mode = 'categorical', # for multi class classification
                                          color_mode='grayscale') # change to single channel

test_dataset = test.flow_from_directory(seed=1, # random seed to maintain same output everytime
                                        directory=test_dataset_path, # path to train set
                                        target_size=(32, 32), # resize to 32x32
                                        batch_size=32, # can be 32, 64, 128
                                        class_mode = 'categorical', # for multi class classification
                                        color_mode='grayscale') # change to single channel

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


## Create the Model

In [4]:
# Define the model
model = keras.Sequential()

# C1: (None,32,32,1) -> (None,28,28,6).
model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(32,32,1), padding='valid'))

# P1: (None,28,28,6) -> (None,14,14,6).
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# C2: (None,14,14,6) -> (None,10,10,16).
model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# P2: (None,10,10,16) -> (None,5,5,16).
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# Flatten: (None,5,5,16) -> (None, 400).
model.add(Flatten())

# FC1: (None, 400) -> (None,120).
model.add(Dense(120, activation='tanh'))

# FC2: (None,120) -> (None,84).
model.add(Dense(84, activation='tanh'))

# FC3: (None,84) -> (None,10).
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) 

INFO:plaidml:Opening device "metal_amd_radeon_pro_5500m.0"


In [5]:
# Get model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_1 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_2 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_2 (Dense)              (None, 84)                10164     
__________

## Train the model

We will train the model for 30 epochs. An epoch is the number of times that the learning algorithms runs through the entire training dataset. Increasing this number will increase the training time.

In [ ]:
# Train the model

# start the timer
start_time = time.time()


history = model.fit_generator(train_dataset,
                              steps_per_epoch=1560, # number of training images/batch size = 50000/32 = 1562
                              epochs=2,
                              validation_data=test_dataset, # validate against the test set on each epoch
                              verbose=2) # print constant updates

# optionally save the model
# model.save('lenet_5_cifar_10.h5')

# stop the timer
time_elapsed = time.time()-start_time
print(f'The model took {time_elapsed}' )

Epoch 1/2
